The power level in a given fuel cell can be found through the following process:

- Find the fuel cell's rack ID, which is its X coordinate plus 10.
- Begin with a power level of the rack ID times the Y coordinate.
- Increase the power level by the value of the grid serial number (your puzzle input).
- Set the power level to itself multiplied by the rack ID.
- Keep only the hundreds digit of the power level (so 12345 becomes 3; numbers with no hundreds digit become 0).
- Subtract 5 from the power level.

### Part 1

In [276]:
import numpy as np
from tqdm import tqdm

In [2]:
SERIAL = 5153

In [303]:
def take_hundred_digit(arr):
    arr = np.where(arr < 100, 0, arr)
    arr = arr // 100
    return arr - 10 * (arr // 10)

def calculate(serial, shape=(300, 300)):
    values = np.zeros(shape)
    yy, xx = np.indices(shape) + 1
    # Find the fuel cell's rack ID, which is its X coordinate plus 10.
    rack_id = xx + 10
    # Begin with a power level of the rack ID times the Y coordinate.
    values = (rack_id * yy)
    # Increase the power level by the value of the grid serial number (your puzzle input).
    values += serial
    # Set the power level to itself multiplied by the rack ID.
    values *= rack_id
    # Keep only the hundreds digit of the power level (so 12345 becomes 3; numbers with no hundreds digit become 0)
    values = take_hundred_digit(values)
    # Subtract 5 from the power level.
    values -= 5
    return values

def find_best_plot(arr, patch_width):
    i_star, j_star, s_star = 0, 0, -1
    for i in range(arr.shape[0]+1-patch_width):
        for j in range(arr.shape[1]+1-patch_width):
            s = arr[i:i+patch_width, j:j+patch_width].sum()
            if s > s_star:
                i_star, j_star, s_star = i, j, s
    return j_star + 1, i_star + 1, s_star

assert(calculate(8)[4, 2] == 4)
assert(calculate(57)[78, 121] == -5)
assert(calculate(39)[195, 216] == 0)
assert(calculate(71)[152, 100] == 4)
assert(find_best_plot(calculate(18), 3)[:2] == (33, 45))
assert(find_best_plot(calculate(42), 3)[:2] == (21, 61))

In [302]:
find_best_plot(calculate(18), 3)[:2]

(33, 45)

### Part 2 (*Sometimes, brute force is good enough*)

In [283]:
from dask.distributed import Client

client = Client()

In [295]:
%%time
import itertools

def find_best_plot_of_size(arr, patch_width):
    return (patch_width, find_best_plot(arr, patch_width))

arr = client.scatter(calculate(5153))
futures = client.map(find_best_plot_of_size, itertools.repeat(arr), range(301))
results = client.gather(futures)

CPU times: user 8.72 s, sys: 4.4 s, total: 13.1 s
Wall time: 1min 21s


In [296]:
from operator import itemgetter
sorted(results, key=lambda x: x[1][2], reverse=True)[:10]

[(12, (236, 227, 110)),
 (14, (236, 225, 107)),
 (15, (236, 224, 106)),
 (16, (236, 223, 100)),
 (13, (236, 226, 97)),
 (18, (236, 223, 91)),
 (17, (236, 223, 89)),
 (11, (237, 228, 88)),
 (19, (236, 222, 81)),
 (10, (236, 227, 78))]